In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from  scipy import stats
from statsmodels.api import qqplot
import statsmodels.api as sm

In [ ]:
from google.colab import files
import pandas as pd
import os 
files.upload()  

In [ ]:
data = pd.read_csv('data_u.csv',usecols=[4])
data = data/100
###2020/01/01 - 2020/04/30###
train = data.head(90)
###2020/05/01 - 2020/05/31###
test = data.tail(31)
train.head(5)

In [ ]:
sm.graphics.tsa.plot_acf(data, lags = 12)
plt.show()
sm.graphics.tsa.plot_pacf(data, lags = 12)
plt.show()
plt.plot(data)
plt.show()

In [ ]:
np.random.seed(123)
###convert int to float###
data = data.values
data = data.astype('float32')
# scaler = MinMaxScaler(feature_range=(0,1))
# data = scaler.fit_transform(data)
train = train.values
train = train.astype('float32')

# scaler = MinMaxScaler(feature_range=(0,1))
# train = scaler.fit_transform(train)

test = test.values
test = test.astype('float32')

# scaler = MinMaxScaler(feature_range=(0,1))
# test = scaler.fit_transform(test)

In [ ]:
def create_dataset(data, look_back = 1):
  dataX,dataY = [],[]
  for i in range(len(data)- look_back):
    a = data[i:(i+look_back), 0]
    dataX.append(a)
    dataY.append(data[i+look_back, 0])
  return np.array(dataX), np.array(dataY)

In [ ]:
look_back = 2
trainX,trainY = create_dataset(train, look_back)
testX,testY = create_dataset(test, look_back)

In [ ]:
trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape =(1,look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(trainX, trainY, epochs = 100, batch_size = 1, verbose = 2)
model.summary()

In [ ]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# trainPredict = scaler.inverse_transform(trainPredict)
# trainY = scaler.inverse_transform([trainY])
# testPredict = scaler.inverse_transform(testPredict)
# testY = scaler.inverse_transform([testY])

# scale inverse
trainPredict = trainPredict*100
trainY = trainY*100
testPredict = testPredict*100
testY = testY*100

In [ ]:
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(trainY, trainPredict[:,0]))
#print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(testY, testPredict[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
#trainPredictPlot = data
# trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
# shift test predictions for plotting
#testPredictPlot = data
# testPredictPlot[:, :] = np.nan
# testPredictPlot[len(trainPredict)+(look_back*2)+1:len(data)-1, :] = testPredict
#plot baseline and predictions
plt.plot(test*100)
plt.plot(testPredict)
#plt.plot(testPredict)
plt.show()